# 水庫資料處理

In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
### get reservoir list from "Statistics" website ###
url = "http://fhy.wra.gov.tw/ReservoirPage_2011/Statistics.aspx"  # 
table = pd.read_html(url)

total_reservoir = list(table[0][0][4:-1])
total_reservoir.remove("集集攔河堰")  # can not grab from the website
total_reservoir.remove('高屏溪攔河堰')  # can not grab from the website

table = table[0]
print(table[table[0] == '鯉魚潭水庫'])

table.index = table[0]
total_reservoir

      0                    1      2      3       4       5        6        7   \
9  鯉魚潭水庫  2019-04-20 23:00:00  18.09  50.93  287.17  300.00  6655.71  58.14 %   

     8   9   10  11    12    13  14  15  16  
9  7.28  --  --  --  0.00  0.30  --  --  --  


['石門水庫',
 '翡翠水庫',
 '寶山第二水庫',
 '永和山水庫',
 '明德水庫',
 '鯉魚潭水庫',
 '德基水庫',
 '石岡壩',
 '霧社水庫',
 '日月潭水庫',
 '湖山水庫',
 '仁義潭水庫',
 '白河水庫',
 '烏山頭水庫',
 '曾文水庫',
 '南化水庫',
 '阿公店水庫',
 '牡丹水庫']

### Test 測試一下

In [ ]:
url = "http://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx"  # website url
table = pd.read_html(url)  # use pandas to read html form
print("type: ", type(table))  # list type
table[0].head()  # table[0] is DataFrame

In [ ]:
'''
欄位內容 :
#### 0: 水庫名稱
#### 1: sum capacity 有效容量(萬立方公尺)
#### 2: time period 起迄時間
#### 3: rain 降雨量(毫米)
#### 4: in water 進水量(萬立方公尺)
#### 5: out water 出水量(萬立方公尺)
#### 6: water level difference 水位差(公尺)
#### 7: time 水情時間
#### 8: current water level 水位(公尺)
#### 9: current water capacity 有效蓄水量(萬立方公尺)
#### 10: water capacity percentage 蓄水百分比(百分比)
'''

## Crawler 網路爬蟲 - 以水庫資料為例
<br>
##### http://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx

In [ ]:
import re
from io import StringIO

In [ ]:
'''
這個函式會找特定的 value ，如「__VIEWSTATE」等
'''
def find_value(name, web):
    reg = 'name="' + name + '".+value="(.*)" />'
    pattern = re.compile(reg)
    result  = pattern.findall(web.text)
    try:
        return result[0]
    except:
        return ""

In [ ]:
'''
處理DataFrame type的資料，並從index list當中獲得要存取的欄位，
加到reservoir_dict_list當中(由不同欄位資料的dictionary組成的list type)，並回傳
'''

In [ ]:
def table_data_management(table, reservoir_dict_list, total_reservoir, index):
    table.index = table[0]  # set column 1 values to be index
    
    for num in range(len(index)):  # scan for each value we want to get(capacity, percentage...)
        for item in total_reservoir:
            value = table.at[item, index[num]]  # access value by reservoir name and column number
            
            try:
                value = float(value)  # try to convert value from string type to float
            except BaseException:  # exception: "--" or "XX.xx%"
                if("%" in value):  # percentage case
                    value = float(value.replace("%", ""))  # remove char %
                else:
                    value = "NULL"  # nan case
                    
            if(item in list(reservoir_dict_list[num].keys())):  # dict[item] has been created
                reservoir_dict_list[num][item].append(value)
            else:  # dict[item] has not been created
                reservoir_dict_list[num][item] = [] # creat a list container
                reservoir_dict_list[num][item].append(value)
                
    return reservoir_dict_list

In [ ]:
'''
因為這個網頁進行爬蟲時，需要 __EVENTTARGET 和 __VIEWSTATE 兩個參數才能順利獲得查詢資料，
而兩個參數會隨每天而有所不同，因此必須先用 GET 的方式存取到兩個參數的值，
接著建立參數們，放入剛剛抓取到的驗證碼及查詢時間格式，
用 POST 方式向網頁抓取資料，最後藉著得到的網頁資訊進行資料的提取。
'''

In [ ]:
def reservoir_data_search(date, reservoir_dict_list, total_reservoir, index):
    # open browser
    ses = requests.Session()

    # enter the website
    d = ses.get('http://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx')
    
    # parameter list
    load_list = [find_value("__EVENTTARGET", d), find_value("__VIEWSTATE", d)]
    if(date[2] == "1"):  # just take a look at schedule
        print("yy/mm/dd:", date)
    
    # website request needs POST parameter
    payload = {
        "__EVENTTARGET": load_list[0],
        "__VIEWSTATE": load_list[1],
        'ctl00$cphMain$cboSearch': "所有水庫",
        'ctl00$cphMain$ucDate$cboYear': date[0],
        'ctl00$cphMain$ucDate$cboMonth': date[1],
        'ctl00$cphMain$ucDate$cboDay': date[2],

    }
    # request to website using POST
    res = requests.post("http://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx", data = payload)
    # manage table data
    reservoir_dict_list = table_data_management(pd.read_html(res.text)[0], reservoir_dict_list, total_reservoir, index)
    
    '''
    如果要用解析html的方法進行資料提取，程式碼為：
    soup = BeautifulSoup(res.text, "lxml").find_all("tr")[2:]  # 從<tr>下手
    reservoir_dict_list = reservoir_data_management(soup, reservoir_dict)
    
    ** 最下面的函式 reservoir_data_management() 是以只存取 percentage 為例 **
    '''
    
    return reservoir_dict_list

In [ ]:
'''
從 dict_list 當中一個個提取欄位的 dictionary，並寫成 csv 檔

parameter:
1) name: 可以自己安排寫檔的名稱

'''

In [ ]:
def write_csv_file(reservoir_dict_list, length, date_index, year):
    name = ["Sum-Capacity", "Out-Daily", "Current-Capacity"]

    for index in range(length):
        final = {}
        for i in reservoir_dict_list[index]:  # check for total data length
            if(len(reservoir_dict_list[index][i]) == len(reservoir_dict_list[index]['石門水庫'])):
                final[i] = reservoir_dict_list[index][i]
        df = pd.DataFrame(final, index = date_index)
        file = name[index] + "-" + str(year) +".csv"  # file name
        df.to_csv(file, encoding="utf_8_sig")  # maintain Chinese word(or they might become garbled)

In [ ]:
'''
透過從2003年到現在日期建立，可以向網頁求得各別每天的資料。

parameter: 
1) year_list: 透過這個年份的賦值，可以指定查詢的年份
2) date_index: 如果日期並不是連續，則可以調整這個 list

'''

In [ ]:
year_list = np.arange(2019,2020)
sum_day = 31
sub_day = [0, -3, 0, -1, 0, -1, 0, 0, -1, 0, -1, 0]  # sum_day + sub_day = month's days

for year in range(len(year_list)):
    current_capacity, out, current_water = {}, {}, {}
    reservoir_dict_list = [current_capacity, out, current_water]  # data contains several columns' value
    date_index = []  # index for DataFrame
    
    for month in range(1,13):
        total = sum_day + sub_day[month-1]
        if(year_list[year] % 4 == 0 and month == 2):  # leap year
            total = total + 1
        for date in range(1, total + 1):
            if(month == 4 and date == 18):
                break
            date_index.append(str(year_list[year]) + "-" +  str(month) + "-" + str(date))
            date_list = [str(year_list[year]), str(month), str(date)]
            index_list = [1, 5, 9]  # columns which we want to get
            reservoir_dict_list = reservoir_data_search(date_list, reservoir_dict_list, total_reservoir, index_list)  #開始parse資料

    print("finish:", year_list[year])  # finish a year
    write_csv_file(reservoir_dict_list, len(index_list), date_index, year_list[year])  # write to csv file

#### others 其它補充

In [ ]:
'''
針對html當中進行資料的提取
(較為複雜，因為要定位該數值在 td 之後的哪個位置，且根據每個水庫名字可能的不同，會有不同的 exception 發生)
最後輸出每個水庫某個欄位數值list的年度(365+1天)dictionary
'''

In [ ]:
def reservoir_data_management(reservoir, reservoir_dict):
    for item in range(len(reservoir)):
        reservoir[item] = str(reservoir[item]).replace("/", "")
        reservoir[item] = reservoir[item].replace("<", "|").replace(">", "|")
        start_index = reservoir[item].index("|td|") + 4
        try:
            end_index = reservoir[item][start_index:start_index+10].index("庫")
        except ValueError: 
            continue
        else:
            end_index = end_index + 1 + start_index

        # find percentage value
        try:
            number_end_index = reservoir[item].index("%") + 1
        except ValueError:
            if(reservoir[item][start_index: end_index] not in reservoir_dict.keys()):
                reservoir_dict[reservoir[item][start_index: end_index]] = ["NULL"]
            else:
                reservoir_dict[reservoir[item][start_index: end_index]].append("NULL")
            continue
        else:
            number_start_index = reservoir[item][number_end_index-10:number_end_index].index("|") - 9 + number_end_index

        value = reservoir[item][number_start_index: number_end_index].replace(" ", "")
        name = reservoir[item][start_index: end_index]
        if(name in list(reservoir_dict.keys())):
            reservoir_dict[name].append(value)
        else:
            reservoir_dict[name] = []
            reservoir_dict[name].append(value)
        if(reservoir[item][start_index: end_index] == "牡丹水庫"):  # terminal reservoir
            break
            
    return reservoir_dict